In [ ]:
import requests
import json
import pprint
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
from   matplotlib.dates    import DateFormatter
from   matplotlib.pyplot   import subplots
from   matplotlib.ticker   import ScalarFormatter
from matplotlib.dates import AutoDateLocator, AutoDateFormatter
import matplotlib.dates as mdates


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/01_Thesis_Python_Operation

/content/drive/MyDrive/01_Thesis_Python_Operation


# **From DAHITI**

TLD country Codes are available at : https://en.wikipedia.org/wiki/Country_code_top-level_domain

## **01 Accessing MetaData**

In [ ]:
os.getcwd()

'/content/drive/MyDrive/01_Thesis_Python_Operation'

In [ ]:
url = 'https://dahiti.dgfi.tum.de/api/v1/'

args = {}
""" required options """
args['username'] = 'tjmahir20'
args['password'] = 'Tj9356301@'
args['action'] = 'list-targets'

""" optional options """
args['basin'] = 'Nile'
args['continent'] = 'Africa'
args['country'] = 'et'
args['min_lon'] = 30
args['max_lon'] = 37
args['min_lat'] = 9
args['max_lat'] = 12
#args['software'] = '3.1'

""" send request as method POST """
response = requests.post(url, data=args)

if response.status_code == 200:
	""" convert json string in python list """
	data = json.loads(response.text)
	pprint.pprint(data)
else:
	print (response.status_code)

[{'access_surface_area': 'denied',
  'access_volume': 'denied',
  'access_water_level': 'request',
  'continent': 'Africa',
  'country': 'et',
  'id': 23335,
  'latitude': 11.0743,
  'location': None,
  'longitude': 35.3781,
  'surface_area': {'last_update': 'None', 'software': None},
  'target_name': 'Beles, River',
  'volume': {'last_update': 'None'},
  'water_level': {'last_update': '2024-01-28 11:36:18', 'software': '8.0'}},
 {'access_surface_area': 'denied',
  'access_volume': 'denied',
  'access_water_level': 'request',
  'continent': 'Africa',
  'country': 'et',
  'id': 17716,
  'latitude': 10.3246,
  'location': None,
  'longitude': 36.6598,
  'surface_area': {'last_update': 'None', 'software': None},
  'target_name': 'Blue Nile, River',
  'volume': {'last_update': 'None'},
  'water_level': {'last_update': '2024-01-28 03:37:33', 'software': '8.0'}},
 {'access_surface_area': 'denied',
  'access_volume': 'denied',
  'access_water_level': 'request',
  'continent': 'Africa',
  'cou

In [ ]:
results = []
for item in data:
    my_dict={}
    my_dict['Dahiti_ID']=item.get('id')
    my_dict['Longitude']=item.get('longitude')
    my_dict['Latitude']=item.get('latitude')
    my_dict['Surface_area']=item.get('surface_area').get("last_update")
    my_dict['Volume']=item.get('volume').get("last_update")
    my_dict['Water_level']=item.get('water_level').get("last_update")

    print (my_dict)
    results.append(my_dict)

{'Dahiti_ID': 23335, 'Longitude': 35.3781, 'Latitude': 11.0743, 'Surface_area': 'None', 'Volume': 'None', 'Water_level': '2024-01-28 11:36:18'}
{'Dahiti_ID': 17716, 'Longitude': 36.6598, 'Latitude': 10.3246, 'Surface_area': 'None', 'Volume': 'None', 'Water_level': '2024-01-28 03:37:33'}
{'Dahiti_ID': 17714, 'Longitude': 36.6171, 'Latitude': 10.3628, 'Surface_area': 'None', 'Volume': 'None', 'Water_level': '2024-01-28 03:37:19'}
{'Dahiti_ID': 17713, 'Longitude': 36.2364, 'Latitude': 10.1289, 'Surface_area': 'None', 'Volume': 'None', 'Water_level': '2024-01-28 03:37:06'}
{'Dahiti_ID': 17711, 'Longitude': 36.0997, 'Latitude': 10.1321, 'Surface_area': 'None', 'Volume': 'None', 'Water_level': '2024-01-28 03:36:56'}
{'Dahiti_ID': 17709, 'Longitude': 35.6052, 'Latitude': 10.018, 'Surface_area': 'None', 'Volume': 'None', 'Water_level': '2024-01-28 03:36:45'}
{'Dahiti_ID': 17708, 'Longitude': 35.5325, 'Latitude': 10.0751, 'Surface_area': 'None', 'Volume': 'None', 'Water_level': '2024-01-28 03:3

In [ ]:
Dahiti_MetaData_DF = pd.DataFrame(results)
Dahiti_MetaData_DF.head()

,Dahiti_ID,Longitude,Latitude,Surface_area,Volume,Water_level
0,23335,35.3781,11.0743,None,None,2024-01-28 11:36:18
1,17716,36.6598,10.3246,None,None,2024-01-28 03:37:33
2,17714,36.6171,10.3628,None,None,2024-01-28 03:37:19
3,17713,36.2364,10.1289,None,None,2024-01-28 03:37:06
4,17711,36.0997,10.1321,None,None,2024-01-28 03:36:56


## **02 Accessing Water Level Timeseries**
Reference link: https://dahiti.dgfi.tum.de/en/api/doc/download-water-level/

In [ ]:
Not_available = []
dahiti_dataframes = []

if not os.path.exists("Data/01_Available_Virtual_Stations/01_DAHITI_Cleaned_Data"):
    os.makedirs("Data/01_Available_Virtual_Stations/01_DAHITI_Cleaned_Data")

for i in Dahiti_MetaData_DF["Dahiti_ID"]:
  url = 'https://dahiti.dgfi.tum.de/api/v1/'

  args = {}
  """ required options """
  args['username'] = 'tjmahir20'
  args['password'] = 'Tj9356301@'
  args['action'] = 'download-water-level'
  args['dahiti_id'] = i

  """ send request as method POST """
  response = requests.post(url, data=args)

  if response.status_code == 200:
    """ convert json string in python list """
    data = json.loads(response.text)
    # pprint.pprint(data)
    Dahiti_DF = pd.DataFrame.from_records(data.get("target").get("data"))
    Dahiti_DF["Dahiti_ID"] = i

    folder_path = "Data/01_Available_Virtual_Stations/01_DAHITI_Cleaned_Data"
    filename = str(i) +"_" + "_DAHITI" + ".csv"
    output_path = os.path.join(folder_path,filename)
    # Dahiti_DF.to_csv(output_path)
    dahiti_dataframes.append(Dahiti_DF)

  else:
    print(f"Data not available for {i} and response code showing {response.status_code}")
    Not_available.append(i)

Data not available for 17716 and response code showing 403
Data not available for 17714 and response code showing 403
Data not available for 17713 and response code showing 403
Data not available for 17711 and response code showing 403
Data not available for 17709 and response code showing 403
Data not available for 17708 and response code showing 403
Data not available for 12013 and response code showing 403


In [ ]:
Dahiti_MetaData_DF["Availability"] = "Available"

# Update the "Availability" column based on the condition
Dahiti_MetaData_DF.loc[Dahiti_MetaData_DF["Dahiti_ID"].isin(Not_available), "Availability"] = "Not Available"

folder_path = "Data/01_Available_Virtual_Stations"
filename = "00_MetaData" + "_DAHITI" + ".csv"
output_path = os.path.join(folder_path,filename)
Dahiti_MetaData_DF.to_csv(output_path)

In [ ]:
dahiti_dataframes[1].head()

,date,height,error,data_type,Dahiti_ID
0,2018-12-08,562.912,0.109791,OP,17706
1,2019-01-04,562.218,0.146950,OP,17706
2,2019-01-31,562.063,0.034783,OP,17706
3,2019-02-27,561.345,0.011483,OP,17706
4,2019-03-26,561.818,0.120746,OP,17706


In [ ]:
Dahiti_MetaData_DF.head()

,Dahiti_ID,Longitude,Latitude,Surface_area,Volume,Water_level,Availability
0,23335,35.3781,11.0743,None,None,2024-01-28 11:36:18,Available
1,17716,36.6598,10.3246,None,None,2024-01-28 03:37:33,Not Available
2,17714,36.6171,10.3628,None,None,2024-01-28 03:37:19,Not Available
3,17713,36.2364,10.1289,None,None,2024-01-28 03:37:06,Not Available
4,17711,36.0997,10.1321,None,None,2024-01-28 03:36:56,Not Available


#**For HydroWeb**

###**01 Downloading Data**

In [ ]:
# pip install eodag

In [ ]:
# workspace = 'eodag_workspace_overview'
# if not os.path.isdir(workspace):
#     os.mkdir(workspace)

In [ ]:
# os.environ["EODAG__HYDROWEB_NEXT__AUTH__CREDENTIALS__APIKEY"] = "UVnOLZFXLSHvxAXstD5vLBDriRZKmz0fY8D1XWbbCct8Iuj3IX"

In [ ]:
# !python 'Data/01_Available_Virtual_Stations/01_download_Theia_Hydroweb_Altimetry_River_Water_Level.py'

In [ ]:
# EODataAccessGateway.update_providers_config(...)

In [ ]:
# For PEPS Credentials
# os.environ["EODAG__PEPS__AUTH__CREDENTIALS__USERNAME"] = "tjmahir20@gmail.com"
# os.environ["EODAG__PEPS__AUTH__CREDENTIALS__PASSWORD"] = "Tazwar9356301@"

### **02 Extracting MetaData from Hydroweb-Downloaded File**

In [ ]:
os.getcwd()

'/content/drive/MyDrive/01_Thesis_Python_Operation'

In [ ]:
path = "Data/01_Available_Virtual_Stations/Hydroweb Stations"
dir_list = os.listdir(path)

In [ ]:
x = os.listdir((os.path.join(path,dir_list[1])))
x

['hydroprd_R_NILE_WHITE-NILE_KM4084_exp.txt']

In [ ]:
Dir = []
Files = []
TxtFileDir = []

folder_path = "Data/01_Available_Virtual_Stations/Hydroweb Stations"
for (root,dirs,files) in os.walk( folder_path, topdown=True):
        # print (root)
        # print (dirs)
        # print (files)
        Dir.append(root)
        Files.append(files)

Dir = Dir[2:]
Files = Files[2:]


for i in range(len(Dir)):
  x = os.path.join(Dir[i],Files[i][0])
  TxtFileDir.append(x)

In [ ]:
Metadata = []
for i in TxtFileDir:
  with open(i) as txt:
      lines = [line for line in txt]
      ColumnNames = [line.split("::")[0] for line in lines [0:25]]
      data = [line.split()[-1] for line in lines [0:25]]
      Metadata.append(data)

In [ ]:
df = pd.DataFrame(columns=ColumnNames)
df

,#BASIN,#RIVER,#ID,#TRIBUTARY OF,#APPROX. WIDTH OF REACH (m),#SURFACE OF UPSTREAM WATERSHED (km2),"#RATING CURVE PARAMETERS A,b,Zo such that Q(m3/s) = A[H(m)-Zo]^b",#REFERENCE ELLIPSOID,#REFERENCE LONGITUDE,#REFERENCE LATITUDE,...,#Bias applied for S6A,#STATUS,#VALIDATION CRITERIA,#MEAN ALTITUDE(M.mm),#MEAN SLOPE (mm/km),#NUMBER OF MEASUREMENTS IN DATASET,#FIRST DATE IN DATASET,#LAST DATE IN DATASET,#DISTANCE MIN IN DATASET (km),#DISTANCE MAX IN DATASET (km)


In [ ]:
for i in range(len(Metadata)):
  # Assuming your new row data is stored in a list
  new_row_data_list = Metadata[i]

  # Convert the list to a DataFrame
  new_row_df = pd.DataFrame([new_row_data_list], columns=df.columns)

  # Concatenate the original DataFrame and the new row DataFrame
  df = pd.concat([df, new_row_df], ignore_index=True)


In [ ]:
df.loc[22]

#BASIN                                                                       NILE
#RIVER                                                                 WHITE-NILE
#ID                                                                 0000000007895
#TRIBUTARY OF                                                        LNYL-TRIB-11
#APPROX. WIDTH OF REACH (m)                                                   210
#SURFACE OF UPSTREAM WATERSHED (km2)                                           NA
#RATING CURVE PARAMETERS A,b,Zo such that Q(m3/s) = A[H(m)-Zo]^b               NA
#REFERENCE ELLIPSOID                                                        WGS84
#REFERENCE LONGITUDE                                                      31.6784
#REFERENCE LATITUDE                                                        9.6106
#REFERENCE DISTANCE (km)                                                     3961
#GEOID MODEL                                                              EGM2008
#GEOID ONDULATIO

In [ ]:
folder_path = "Data/01_Available_Virtual_Stations"
df.to_csv(os.path.join(folder_path,"00_MetaData_Hydroweb.csv"))

### **03 Extracting Water Level Timeseries**

In [ ]:
if not os.path.exists("Data/01_Available_Virtual_Stations/01_Hydroweb_Cleaned_Data"):
    os.makedirs("Data/01_Available_Virtual_Stations/01_Hydroweb_Cleaned_Data")

In [ ]:
Dir = []
Files = []
TxtFileDir = []

folder_path = "Data/01_Available_Virtual_Stations/Hydroweb Stations"
for (root,dirs,files) in os.walk( folder_path, topdown=True):
        # print (root)
        # print (dirs)
        # print (files)
        Dir.append(root)
        Files.append(files)

Dir = Dir[2:]
Files = Files[2:]


for i in range(len(Dir)):
  x = os.path.join(Dir[i],Files[i][0])
  TxtFileDir.append(x)

In [ ]:
Columns = []
for i in TxtFileDir:
  with open(i) as txt:
      lines = [line for line in txt]
      Columns = [line.split(":")[-1] for line in lines [30:46]]

In [ ]:
folder_path = "Data/01_Available_Virtual_Stations/01_Hydroweb_Cleaned_Data"

In [ ]:
exceptionals_staions = []
hydroweb_dataframes = []

for i in range(len(TxtFileDir)):
  with open(TxtFileDir[i]) as txt:
      lines = [line for line in txt]
      Columns = [line.split(":")[-1] for line in lines [28:44]]
      Data = [line.split() for line in lines [45:]]

      if len(Columns) == len(Data[0]):
        XF = pd.DataFrame(columns=Columns)

        for j in range(len(Data)):
          new_row =  Data[j]
          # Convert the list to a DataFrame
          new_row_XF = pd.DataFrame([new_row], columns=XF.columns)
          # Concatenate the original DataFrame and the new row DataFrame
          XF = pd.concat([XF, new_row_XF], ignore_index=True)

        XF["Hydroweb_ID"] = df["#ID"][i]
        filename = df["#RIVER"][i] +"_"+ df["#ID"][i] + "_Hydroweb" + ".csv"
        output_path = os.path.join(folder_path,filename)
        XF.to_csv(output_path)
        hydroweb_dataframes.append(XF)

      else:
        print(i)
        exceptionals_staions.append(i)


  #       # Assuming your new row data is stored in a list
  # new_row_data_list = Metadata[i]

  # # Convert the list to a DataFrame
  # new_row_df = pd.DataFrame([new_row_data_list], columns=df.columns)

  # # Concatenate the original DataFrame and the new row DataFrame
  # df = pd.concat([df, new_row_df], ignore_index=True)

35
40


In [ ]:
hydroweb_dataframes[1]["Hydroweb_ID"].head()

0    0000000007837
1    0000000007837
2    0000000007837
3    0000000007837
4    0000000007837
Name: Hydroweb_ID, dtype: object

In [ ]:
XF.loc[1]

 DATE(YYYY-MM-DD)\n                                                    2019-04-12
MM)\n                                                                       07:43
 ORTHOMETRIC HEIGHT (M) OF WATER SURFACE AT REFERENCE POSITION\n           613.62
 ASSOCIATED UNCERTAINTY(M)\n                                                 0.19
\n                                                                              :
 LONGITUDE OF ALTIMETRY MEASUREMENT (deg)\n                              9999.999
 LATITUDE OF ALTIMETRY MEASUREMENT (deg)\n                               9999.999
 ELLIPSOIDAL HEIGHT OF ALTIMETRY MEASUREMENT (M)\n                         606.05
 GEOIDAL ONDULATION (M) at location [5,6]\n                                 -7.57
 DISTANCE OF ALTIMETRY MEASUREMENT TO REFERENCE POSITION(KM)\n           9999.999
 SATELLITE\n                                                                  S3A
 ORBIT / MISSION\n                                                            REP
 GROUND-TRACK NU

### **04 Dealing with Exceptional Stations**

### In the raw data, some stations might have different data formations. This previous code cell will identify and work on similarly formatted virtual stations. It will identify dissimilarly formatted stations as ***Exceptional Stations***. The user needs to manually check those data in order to extract the required data.

In [ ]:
exceptionals_staions

[35, 40]

In [ ]:
for i in exceptionals_staions:
  with open(TxtFileDir[i]) as txt:
    lines = [line for line in txt]
    Columns = [line.split(":")[-1] for line in lines [30:46]] ### Need to Manually Adjust the Index values by observing the Txt file
    Data = [line.split() for line in lines [47:]]             ### Need to Manually Adjust the Index values by observing the Txt file

    if len(Columns) == len(Data[0]):
        XF = pd.DataFrame(columns=Columns)

        for j in range(len(Data)):
          new_row =  Data[j]
          # Convert the list to a DataFrame
          new_row_XF = pd.DataFrame([new_row], columns=XF.columns)
          # Concatenate the original DataFrame and the new row DataFrame
          XF = pd.concat([XF, new_row_XF], ignore_index=True)
        XF["Hydroweb_ID"] = df["#ID"][i]
        filename = df["#RIVER"][i] +"_"+ df["#ID"][i] + "_Hydroweb" + ".csv"
        output_path = os.path.join(folder_path,filename)
        XF.to_csv(output_path)
        hydroweb_dataframes.append(XF)